# Submission Machine Learning Pipeline

| Informasi                | Deskripsi                                                                                                                                                                                                 |
|--------------------------|-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **Dataset**              | **Nama Dataset:**  House Sales Dataset<br>**Jumlah data:** 21,613 baris<br>**Jumlah fitur:** 16 kolom setelah preprocessing (menghapus kolom id, date, zipcode, lat, dan long)                      |
| **Persoalan**            | Prediksi harga rumah berdasarkan fitur-fitur seperti jumlah kamar tidur, jumlah kamar mandi, luas bangunan, luas tanah, tahun renovasi, dll.                                                              |
| **Solusi Machine Learning** | Membuat model prediksi berbasis supervised learning menggunakan Neural Network.<br>**Target:** Menurunkan Mean Absolute Error (MAE) serendah mungkin untuk meningkatkan akurasi prediksi harga rumah. |
| **Metode Pengolahan Data** | - Menghapus kolom yang tidak relevan (id, date, zipcode, lat, long).<br>- Memastikan tidak ada nilai kosong dalam dataset.<br>- Normalisasi data numerik untuk memastikan keseragaman skala fitur.     |
| **Arsitektur Model**     | **Jenis Model:** Neural Network<br>**Arsitektur:**<br>- Input layer: 15 fitur<br>- Hidden layers: 2-3 dense layers dengan fungsi aktivasi ReLU<br>- Output layer: 1 neuron dengan fungsi aktivasi linear<br>- Optimizer: Adam<br>- Loss Function: Mean Squared Error |
| **Metrik Evaluasi**      | Mean Absolute Error (MAE) digunakan untuk mengevaluasi performa model.                                                                                                                                   |
| **Performa Model**       | **Hasil evaluasi model selama 10 epoch:**                                                                                                                                                                 |

| Epoch | Loss             | MAE       | Val Loss         | Val MAE    |
|-------|------------------|-----------|------------------|------------|
| 1     | 145,715,298,304.0| 252,017.48| 37,374,951,424.0 | 127,390.52 |
| 2     | 40,713,646,080.0 | 131,464.44| 34,311,292,928.0 | 125,720.11 |
| 3     | 39,546,019,840.0 | 129,368.49| 34,456,485,888.0 | 125,261.94 |
| 4     | 38,072,541,184.0 | 127,328.05| 32,720,058,368.0 | 122,249.05 |
| 5     | 37,539,704,832.0 | 126,446.96| 32,377,499,648.0 | 122,781.50 |
| 6     | 35,980,206,080.0 | 124,723.56| 32,648,998,912.0 | 122,197.02 |
| 7     | 36,111,826,944.0 | 124,676.31| 31,019,323,392.0 | 120,563.74 |
| 8     | 34,755,887,104.0 | 122,434.11| 30,812,911,616.0 | 121,035.16 |
| 9     | 34,431,885,312.0 | 122,069.17| 30,086,307,840.0 | 120,127.05 |
| 10    | 34,304,559,104.0 | 121,650.51| 29,608,245,248.0 | 118,439.11 |


# Tujuan: Mengimpor library yang diperlukan untuk interaksi IPython dan menampilkan output, serta mengupgrade pip jika kode dijalankan di Google Colab.

In [1]:
try:
  import colab
  !pip install --upgrade pip
except:
  pass

# Menginstal atau mengupdate library TFX (TensorFlow Extended)


In [2]:
!pip install -U tfx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.7/89.7 kB 5.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.0/152.0 kB 13.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.5/173.5 kB 16.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.9/99.9 kB 11.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of tf-keras to determine which version is compatible with other requirements. This could take a while.
I

#  Mengimpor TensorFlow dan TFX, kemudian mencetak versi masing-masing.

In [3]:
import tensorflow as tf
print('TensorFlow version: {}'.format(tf.__version__))
from tfx import v1 as tfx
print('TFX version: {}'.format(tfx.__version__))

TensorFlow version: 2.16.2
TFX version: 1.16.0


# Menginstal library tambahan, mengimpor library dan modul yang diperlukan untuk pipeline TFX.

In [4]:
%pip install tfx tensorflow_model_analysis

import os
import pandas as pd
import tensorflow as tf
from tfx.components import CsvExampleGen, StatisticsGen, SchemaGen, ExampleValidator, Transform, Trainer, Tuner
from tfx.proto import example_gen_pb2
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
import tensorflow_model_analysis as tfma

## Set Variable

# Load Data
Sel ini mengunduh dataset dari URL yang ditentukan dan menyimpannya ke file sementara.

In [5]:
import urllib.request
import tempfile

DATA_ROOT = tempfile.mkdtemp(prefix='tfx-data')  # Create a temporary directory.
_data_url = 'https://assets.datacamp.com/production/repositories/3953/datasets/818cd72f8f36fb710e52074a41b0cfd322915eaa/kc_house_data.csv'
_data_filepath = os.path.join(DATA_ROOT, "data.csv")
urllib.request.urlretrieve(_data_url, _data_filepath)

('/tmp/tfx-data_h_pw6rb/data.csv', <http.client.HTTPMessage at 0x7acebd8729b0>)

# Initialization Variable
## Sel ini mendefinisikan beberapa variabel, yang akan digunakan di seluruh pipeline

In [6]:
# Setting pipeline and schema names
PIPELINE_NAME = "house-sales-pipeline"
SCHEMA_PIPELINE_NAME = "house-sales-tfdv-schema"

# Defining paths for pipeline artifacts, metadata, and serving model
PIPELINE_ROOT = os.path.join('pipelines', PIPELINE_NAME)  # Directory for storing pipeline artifacts
METADATA_PATH = os.path.join('metadata', PIPELINE_NAME, 'metadata.db')  # SQLite metadata storage path
SERVING_MODEL_DIR = os.path.join('serving_model', PIPELINE_NAME)  # Directory to save exported models

# Load the stress dataset
house_sales_df = _data_url



# Sel ini membaca data yang diunduh menggunakan pandas, menampilkan beberapa baris pertama, dan mencetak informasi tentang dataframe.

In [7]:
import pandas as pd
!head {_data_filepath}
data_url = pd.read_csv(_data_filepath)
data_url.head()
data_url.info()

id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
"7129300520","20141013T000000",221900,3,1,1180,5650,"1",0,0,3,7,1180,0,1955,0,"98178",47.5112,-122.257,1340,5650
"6414100192","20141209T000000",538000,3,2.25,2570,7242,"2",0,0,3,7,2170,400,1951,1991,"98125",47.721,-122.319,1690,7639
"5631500400","20150225T000000",180000,2,1,770,10000,"1",0,0,3,6,770,0,1933,0,"98028",47.7379,-122.233,2720,8062
"2487200875","20141209T000000",604000,4,3,1960,5000,"1",0,0,5,7,1050,910,1965,0,"98136",47.5208,-122.393,1360,5000
"1954400510","20150218T000000",510000,3,2,1680,8080,"1",0,0,3,8,1680,0,1987,0,"98074",47.6168,-122.045,1800,7503
"7237550310","20140512T000000",1.225e+006,4,4.5,5420,101930,"1",0,0,3,11,3890,1530,2001,0,"98053",47.6561,-122.005,4760,101930
"1321400060","20140627T000000",257500,3,2.25,1715,6819,"2",0,0,3,7,1715,0,1995,0,"98003",47.3097,-122.327,2238,6819
"2

# Data Cleaning
##Sel ini memproses data dengan menghapus kolom yang tidak perlu dan menyimpan data yang diproses ke file CSV baru.

In [8]:
import os

# Define the directory to save the cleaned dataset
data_path = "data"

# Create the directory if it doesn't exist
os.makedirs(data_path, exist_ok=True)

# List of columns to drop (irrelevant or redundant for regression)
columns_to_drop = [
    "id",              # Identifier, not useful for regression
    "date",            # Needs transformation if used (e.g., year_sold)
    "zipcode",         # Could be encoded or aggregated into neighborhood
    "lat", "long"      # Optional: Transform to features like distance from city center
]

# Filter columns_to_drop to only include existing columns
existing_columns = [col for col in columns_to_drop if col in data_url.columns]

# Output the columns that will be dropped
if existing_columns:
    print(f"Columns to drop: {existing_columns}")
    data_url = data_url.drop(columns=existing_columns)
    print("Columns dropped successfully.")
else:
    print("No columns to drop.")

# Check if the DataFrame is empty after dropping columns
if data_url.empty:
    print("Warning: The DataFrame is empty after dropping columns.")
else:
    print(f"DataFrame shape after dropping columns: {data_url.shape}")

# Save the cleaned dataset to a new CSV file
output_path = os.path.join(data_path, "cleaned_house_sales_dataset.csv")
data_url.to_csv(output_path, index=False)

print(f"Cleaned dataset saved to: {output_path}")
data_url.head()
data_url.info()


Columns to drop: ['id', 'date', 'zipcode', 'lat', 'long']
Columns dropped successfully.
DataFrame shape after dropping columns: (21613, 16)
Cleaned dataset saved to: data/cleaned_house_sales_dataset.csv
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21613 entries, 0 to 21612
Data columns (total 16 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   price          21613 non-null  float64
 1   bedrooms       21613 non-null  int64  
 2   bathrooms      21613 non-null  float64
 3   sqft_living    21613 non-null  int64  
 4   sqft_lot       21613 non-null  int64  
 5   floors         21613 non-null  float64
 6   waterfront     21613 non-null  int64  
 7   view           21613 non-null  int64  
 8   condition      21613 non-null  int64  
 9   grade          21613 non-null  int64  
 10  sqft_above     21613 non-null  int64  
 11  sqft_basement  21613 non-null  int64  
 12  yr_built       21613 non-null  int64  
 13  yr_renovated   21613 no

# Data Preparation
## Sel ini menginisialisasi InteractiveContext, yang memungkinkan eksekusi interaktif dari pipeline TFX.

In [9]:
DATA_ROOT = 'data'  # Define the root directory for data

# Initialize the interactive context for TFX pipeline
interactive_context = InteractiveContext(pipeline_root=PIPELINE_ROOT)

# Data Ingestion
# Example Gen
## Sel ini membuat komponen example_gen yang merupakan langkah pertama dalam pipeline TFX. Komponen example_gen memasukkan data.

In [10]:
output = example_gen_pb2.Output(
    split_config = example_gen_pb2.SplitConfig(splits = [
        example_gen_pb2.SplitConfig.Split(name = "train", hash_buckets = 8),
        example_gen_pb2.SplitConfig.Split(name = "eval", hash_buckets = 2)
    ])
)

example_gen = CsvExampleGen(input_base = DATA_ROOT, output_config = output)

# Sel ini menjalankan komponen example_gen dalam konteks interaktif.

In [11]:
interactive_context.run(example_gen)

ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 1
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=CsvExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

# Data Validation
# Statistic Gen
## membuat komponen StatisticsGen yang akan bekerja pada data yang telah dimasukkan oleh example_gen dan kemudian kita jalankan komponen tersebut. [1, 2]

In [12]:
statistic_gen = StatisticsGen(
    examples = example_gen.outputs["examples"]
)

interactive_context.run(statistic_gen)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 2
    outputs:
        statistics: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

# Kode ini menampilkan hasil statistik yang telah dihitung oleh StatisticsGen. Hasilnya biasanya berupa tabel atau visualisasi yang memberikan gambaran tentang data Anda.

In [13]:
interactive_context.show(statistic_gen.outputs['statistics'])

# Schema Gen
## Membuat komponen SchemaGen. Komponen ini menggunakan statistik yang dihasilkan oleh StatisticsGen untuk membuat skema data. Skema ini mendefinisikan tipe data, validasi, dan properti lain dari setiap fitur dalam dataset Anda. Skema ini penting untuk memastikan konsistensi dan kualitas data di seluruh pipeline TFX. Komponen SchemaGen dijalankan setelah dibuat.

In [14]:
schema_gen = SchemaGen(statistics = statistic_gen.outputs["statistics"])

interactive_context.run(schema_gen)

ExecutionResult(
    component_id: SchemaGen
    execution_id: 3
    outputs:
        schema: OutputChannel(artifact_type=Schema, producer_component_id=SchemaGen, output_key=schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

# Kode ini menampilkan skema yang telah dihasilkan oleh SchemaGen. Skema ini dapat digunakan oleh komponen lain dalam pipeline TFX, misalnya untuk validasi data.

In [15]:
interactive_context.show(schema_gen.outputs["schema"])

,Type,Presence,Valency,Domain
Feature name,,,,
'bathrooms',FLOAT,required,,-
'bedrooms',INT,required,,-
'condition',INT,required,,-
'floors',FLOAT,required,,-
'grade',INT,required,,-
'price',FLOAT,required,,-
'sqft_above',INT,required,,-
'sqft_basement',INT,required,,-
'sqft_living',INT,required,,-


# Example Validator

In [16]:
example_validator = ExampleValidator(
    statistics = statistic_gen.outputs["statistics"],
    schema = schema_gen.outputs["schema"]
)

interactive_context.run(example_validator)

ExecutionResult(
    component_id: ExampleValidator
    execution_id: 4
    outputs:
        anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=ExampleValidator, output_key=anomalies, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

In [17]:
interactive_context.show(example_validator.outputs["anomalies"])

# Data Preprocessing
## The Transform module is responsible for feature preprocessing, transforming the dataset, and preparing it for subsequent steps in the pipeline. This includes renaming features with a specific naming convention, converting data types, and applying transformations to features as required.

In [18]:
# Define the file containing transformation logic
TRANSFORM_MODULE_FILE = "house_sales_transform.py"

In [19]:
%%writefile house_sales_transform.py

# Import required libraries for TensorFlow, TensorFlow Transform, Keras, and TFX utilities
import tensorflow as tf
import tensorflow_transform as tft
from tensorflow.keras import layers
import os
from tfx.components.trainer.fn_args_utils import FnArgs

# Define constants for label and feature keys
LABEL_KEY = "price"
FEATURE_KEYS = [
    "bedrooms", "bathrooms", "sqft_living", "sqft_lot", "floors", "waterfront",
    "view", "condition", "grade", "sqft_above", "sqft_basement", "yr_built",
    "yr_renovated", "sqft_living15", "sqft_lot15"
]

# Rename transformed features to append '_xf' to the original feature names
def transformed_name(key):
    return key + "_xf"

# Function to read compressed TFRecord files
def gzip_reader_fn(filenames):
    return tf.data.TFRecordDataset(filenames, compression_type='GZIP')

# Create a batched dataset from transformed features
def input_fn(file_pattern, tf_transform_output, num_epochs, batch_size=64) -> tf.data.Dataset:
    # Load transformed feature specification from TensorFlow Transform
    transform_feature_spec = tf_transform_output.transformed_feature_spec().copy()

    # Create a batched dataset with specified features, labels, and batch size
    dataset = tf.data.experimental.make_batched_features_dataset(
        file_pattern=file_pattern,
        batch_size=batch_size,
        features=transform_feature_spec,
        reader=gzip_reader_fn,
        num_epochs=num_epochs,
        label_key=transformed_name(LABEL_KEY)
    )
    return dataset

# Build the deep learning model for regression (predicting price)
def model_builder():
    # Define input layer for numerical features (transformed feature)
    inputs = {feature: tf.keras.Input(shape=(1,), name=transformed_name(feature), dtype=tf.float32) for feature in FEATURE_KEYS}
    concatenated = layers.Concatenate()(list(inputs.values()))

    # Add fully connected (dense) layers
    x = layers.Dense(128, activation="relu")(concatenated)
    x = layers.Dense(64, activation="relu")(x)
    x = layers.Dense(32, activation="relu")(x)

    # Output layer for regression (no activation function, as we are predicting a continuous value)
    outputs = layers.Dense(1)(x)

    # Compile the model with mean squared error loss (for regression)
    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    model.compile(
        loss='mean_squared_error',
        optimizer=tf.keras.optimizers.Adam(0.01),
        metrics=['mean_absolute_error']
    )

    # Print model summary
    model.summary()
    return model

# Define a function to preprocess raw request data for deployment
def _get_serve_tf_examples_fn(model, tf_transform_output):
    # Attach TFT transform features layer to the model
    model.tft_layer = tf_transform_output.transform_features_layer()

    @tf.function
    def serve_tf_examples_fn(serialized_tf_examples):
        # Parse raw features and apply transformations
        feature_spec = tf_transform_output.raw_feature_spec()
        feature_spec.pop(LABEL_KEY)  # Remove label key from feature spec
        parsed_features = tf.io.parse_example(serialized_tf_examples, feature_spec)
        transformed_features = model.tft_layer(parsed_features)
        return model(transformed_features)

    return serve_tf_examples_fn

# Define a function to get the filepath with .keras extension
def get_keras_filepath(model_dir):
  return os.path.join(model_dir, 'model.keras')

# Define the main function to execute training and save the model
def run_fn(fn_args: FnArgs) -> None:
    # Specify directory for TensorBoard logs
    log_dir = os.path.join(os.path.dirname(fn_args.serving_model_dir), 'logs')

    # Configure TensorBoard callback
    tensorboard_callback = tf.keras.callbacks.TensorBoard(
        log_dir=log_dir, update_freq='batch', profile_batch=0  # Add profile_batch=0
    )

    # Early stopping callback to prevent overfitting
    es = tf.keras.callbacks.EarlyStopping(
        monitor='val_mean_absolute_error', mode='min', verbose=1, patience=10
    )

    # Get filepath with .keras extension
    filepath = get_keras_filepath(fn_args.serving_model_dir)

    # Model checkpoint callback to save the best model
    mc = tf.keras.callbacks.ModelCheckpoint(
        filepath, monitor='val_mean_absolute_error',
        mode='min', verbose=1, save_best_only=True
    )

    # Load the transform graph output for data transformation
    tf_transform_output = tft.TFTransformOutput(fn_args.transform_graph_path)

    # Create training and validation datasets
    train_set = input_fn(fn_args.train_files, tf_transform_output, num_epochs=10)
    val_set = input_fn(fn_args.eval_files, tf_transform_output, num_epochs=10)

    # Build the model
    model = model_builder()

    # Train the model with training and validation datasets
    model.fit(
        x=train_set,
        validation_data=val_set,
        callbacks=[tensorboard_callback, es, mc],
        steps_per_epoch=1000,
        validation_steps=1000,
        epochs=10
    )

    # Define model signatures for serving
    signatures = {
        'serving_default': _get_serve_tf_examples_fn(model, tf_transform_output).get_concrete_function(
            tf.TensorSpec(
                shape=[None], dtype=tf.string, name='examples'
            )
        )
    }

    # Save the trained model for serving
    tf.saved_model.save(model, fn_args.serving_model_dir, signatures=signatures)


# Define the preprocessing function for feature scaling using z-score
def preprocessing_fn(inputs):
    """tf.transform's callback function for preprocessing inputs.
    Args:
        inputs: map from feature keys to raw not-yet-transformed features.
    Returns:
        Map from string feature key to transformed feature.
    """
    outputs = {}
    for key in FEATURE_KEYS:
        # Scale numerical features to have zero mean and unit variance
        outputs[transformed_name(key)] = tft.scale_to_z_score(inputs[key])

    # Keep the label
    outputs[transformed_name(LABEL_KEY)] = inputs[LABEL_KEY]
    return outputs

Writing house_sales_transform.py


# Transform

In [20]:
from tfx.components import Transform
import os

TRANSFORM_MODULE_FILE = "house_sales_transform.py"  # Path to the transformation logic file

transform = Transform(
    examples=example_gen.outputs["examples"],  # Input examples from ExampleGen
    schema=schema_gen.outputs["schema"],      # Schema generated by SchemaGen
    module_file=os.path.abspath(TRANSFORM_MODULE_FILE)  # Absolute path to transform module
)

In [21]:
interactive_context.run(transform)

ExecutionResult(
    component_id: Transform
    execution_id: 5
    outputs:
        transform_graph: OutputChannel(artifact_type=TransformGraph, producer_component_id=Transform, output_key=transform_graph, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        transformed_examples: OutputChannel(artifact_type=Examples, producer_component_id=Transform, output_key=transformed_examples, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        updated_analyzer_cache: OutputChannel(artifact_type=TransformCache, producer_component_id=Transform, output_key=updated_analyzer_cache, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        pre_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=pre_transform_schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        pre_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=pre_transform_stats, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        post_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=post_transform_schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        post_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=post_transform_stats, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        post_transform_anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=Transform, output_key=post_transform_anomalies, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

# Training Model

In [22]:
TRAINER_MODULE_FILE = "house_sales_trainer.py"

In [23]:
%%writefile house_sales_trainer.py
import tensorflow as tf
import tensorflow_transform as tft
from tensorflow.keras import layers
import os
from tfx.components.trainer.fn_args_utils import FnArgs

# Define constants for label and feature keys
LABEL_KEY = "price"
FEATURE_KEYS = ["bedrooms", "bathrooms", "sqft_living", "sqft_lot", "floors", "waterfront", "view", "condition", "grade",
                "sqft_above", "sqft_basement", "yr_built", "yr_renovated", "sqft_living15", "sqft_lot15"]

# Rename transformed features to append '_xf' to the original feature names
def transformed_name(key):
    return key + "_xf"

# Function to read compressed TFRecord files
def gzip_reader_fn(filenames):
    return tf.data.TFRecordDataset(filenames, compression_type='GZIP')

# Create a batched dataset from transformed features
def input_fn(file_pattern, tf_transform_output, num_epochs, batch_size=64) -> tf.data.Dataset:
    # Load transformed feature specification from TensorFlow Transform
    transform_feature_spec = tf_transform_output.transformed_feature_spec().copy()

    # Create a batched dataset with specified features, labels, and batch size
    dataset = tf.data.experimental.make_batched_features_dataset(
        file_pattern=file_pattern,
        batch_size=batch_size,
        features=transform_feature_spec,
        reader=gzip_reader_fn,
        num_epochs=num_epochs,
        label_key=transformed_name(LABEL_KEY)  # Use transformed label key
    )
    return dataset

# Build the deep learning model for regression (predicting price)
def model_builder():
    # Define input layer for numerical features (transformed feature)
    inputs = {transformed_name(feature): tf.keras.Input(shape=(1,), name=transformed_name(feature), dtype=tf.float32)
              for feature in FEATURE_KEYS}
    concatenated = layers.Concatenate()(list(inputs.values()))

    # Add fully connected (dense) layers
    x = layers.Dense(128, activation="relu")(concatenated)
    x = layers.Dense(64, activation="relu")(x)
    x = layers.Dense(32, activation="relu")(x)

    # Output layer for regression (no activation function, as we are predicting a continuous value)
    outputs = layers.Dense(1)(x)

    # Compile the model with mean squared error loss (for regression)
    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    model.compile(
        loss='mean_squared_error',
        optimizer=tf.keras.optimizers.Adam(0.01),
        metrics=['mean_absolute_error']
    )

    # Print model summary
    model.summary()
    return model

# Function to preprocess raw request data for deployment
def _get_serve_tf_examples_fn(model, tf_transform_output):
    # Attach TFT transform features layer to the model
    model.tft_layer = tf_transform_output.transform_features_layer()

    @tf.function
    def serve_tf_examples_fn(serialized_tf_examples):
        # Parse raw features and apply transformations
        feature_spec = tf_transform_output.raw_feature_spec()
        # feature_spec.pop(LABEL_KEY)  # Remove label key from feature spec - DO NOT POP
        parsed_features = tf.io.parse_example(serialized_tf_examples, feature_spec)
        transformed_features = model.tft_layer(parsed_features)
        return model(transformed_features)

    return serve_tf_examples_fn

# Function to define filepath for saving the model
def get_keras_filepath(model_dir):
    return os.path.join(model_dir, 'model.keras')

# Main function for training
def run_fn(fn_args: FnArgs) -> None:
    # Load the transform graph output for data transformation
    tf_transform_output = tft.TFTransformOutput(fn_args.transform_graph_path)

    # Create training and validation datasets
    train_set = input_fn(fn_args.train_files, tf_transform_output, num_epochs=10)
    val_set = input_fn(fn_args.eval_files, tf_transform_output, num_epochs=10)

    # Define TensorBoard callback
    log_dir = os.path.join(os.path.dirname(fn_args.serving_model_dir), 'logs')
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, update_freq='batch', profile_batch=0)

    # Early stopping callback
    es = tf.keras.callbacks.EarlyStopping(monitor='val_mean_absolute_error', mode='min', verbose=1, patience=10)

    # Model checkpoint callback
    filepath = get_keras_filepath(fn_args.serving_model_dir)
    mc = tf.keras.callbacks.ModelCheckpoint(filepath, monitor='val_mean_absolute_error',
                                            mode='min', verbose=1, save_best_only=True)

    # Build the model
    model = model_builder()

    # Train the model
    model.fit(
        x=train_set,
        validation_data=val_set,
        callbacks=[tensorboard_callback, es, mc],
        steps_per_epoch=1000,
        validation_steps=1000,
        epochs=10
    )

    # Define model signatures for serving
    signatures = {
        'serving_default': _get_serve_tf_examples_fn(model, tf_transform_output).get_concrete_function(
            tf.TensorSpec(shape=[None], dtype=tf.string, name='examples')
        )
    }

    # Save the model
    tf.saved_model.save(model, fn_args.serving_model_dir, signatures=signatures)


Writing house_sales_trainer.py


In [24]:
# Create the Trainer component to train the model
trainer = Trainer(
    module_file=os.path.abspath(TRAINER_MODULE_FILE),  # Path to training logic
    examples=transform.outputs["transformed_examples"],  # Transformed data
    transform_graph=transform.outputs["transform_graph"],  # Transformation graph
    schema=schema_gen.outputs["schema"],  # Data schema
    # hyperparameters=tuner.outputs["best_hyperparameters"],  # Best hyperparameters from Tuner
    # hyperparameters=hyperparameter_tuner.outputs["best_hyperparameters"],  # Best hyperparameters from Tuner instance
    train_args=trainer_pb2.TrainArgs(splits=["train"], num_steps=800),  # Training config
    eval_args=trainer_pb2.EvalArgs(splits=["eval"], num_steps=400),  # Evaluation config
)

# Run the Trainer component in an interactive environment
interactive_context.run(trainer)

Instructions for updating:
Use `tf.data.Dataset.map(tf.io.parse_example(...))` instead.


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ bedrooms_xf (InputLayer)  │ (None, 1)              │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ bathrooms_xf (InputLayer) │ (None, 1)              │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ sqft_living_xf            │ (None, 1)              │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ sqft_lot_xf (InputLayer)  │ (None, 1)              │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ floors_xf (InputLayer)    │ (None, 1)              │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ waterfront_xf             │ (None, 1)              │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ view_xf (InputLayer)      │ (None, 1)              │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ condition_xf (InputLayer) │ (None, 1)              │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ grade_xf (InputLayer)     │ (None, 1)              │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ sqft_above_xf             │ (None, 1)              │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ sqft_basement_xf          │ (None, 1)              │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ yr_built_xf (InputLayer)  │ (None, 1)              │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ yr_renovated_xf           │ (None, 1)              │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ sqft_living15_xf          │ (None, 1)              │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ sqft_lot15_xf             │ (None, 1)              │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate (Concatenate) │ (None, 15)             │              0 │ bedrooms_xf[0][0],     │
│                      

 Total params: 12,417 (48.50 KB)

 Trainable params: 12,417 (48.50 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
 994/1000 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 146231377920.0000 - mean_absolute_error: 252594.4375

/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)



Epoch 1: val_mean_absolute_error improved from inf to 127390.52344, saving model to pipelines/house-sales-pipeline/Trainer/model/6/Format-Serving/model.keras
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 18s 12ms/step - loss: 145715298304.0000 - mean_absolute_error: 252017.4844 - val_loss: 37374951424.0000 - val_mean_absolute_error: 127390.5234
Epoch 2/10
 995/1000 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 40713646080.0000 - mean_absolute_error: 131464.4375
Epoch 2: val_mean_absolute_error improved from 127390.52344 to 125720.10938, saving model to pipelines/house-sales-pipeline/Trainer/model/6/Format-Serving/model.keras
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 7s 7ms/step - loss: 40708628480.0000 - mean_absolute_error: 131458.2031 - val_loss: 34311292928.0000 - val_mean_absolute_error: 125720.1094
Epoch 3/10
 701/1000 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - loss: 39932329984.0000 - mean_absolute_error: 129691.0078
Epoch 3: val_mean_absolute_error improved from 125720.10938 to 125261.93750, saving model to pipeline

ExecutionResult(
    component_id: Trainer
    execution_id: 6
    outputs:
        model: OutputChannel(artifact_type=Model, producer_component_id=Trainer, output_key=model, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        model_run: OutputChannel(artifact_type=ModelRun, producer_component_id=Trainer, output_key=model_run, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

# Resolver

In [25]:
from tfx.dsl.components.common.resolver import Resolver
from tfx.dsl.input_resolution.strategies.latest_blessed_model_strategy import LatestBlessedModelStrategy
from tfx.types import Channel
from tfx.types.standard_artifacts import Model, ModelBlessing

model_resolver = Resolver(
    strategy_class = LatestBlessedModelStrategy,
    model = Channel(type = Model),
    model_blessing = Channel(type = ModelBlessing)
).with_id('Latest_blessed_model_resolver')

interactive_context.run(model_resolver)

ExecutionResult(
    component_id: Latest_blessed_model_resolver
    execution_id: 7
    outputs:
        model: OutputChannel(artifact_type=Model, producer_component_id=Latest_blessed_model_resolver, output_key=model, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        model_blessing: OutputChannel(artifact_type=ModelBlessing, producer_component_id=Latest_blessed_model_resolver, output_key=model_blessing, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

# Evaluator

In [27]:
import tensorflow_model_analysis as tfma

eval_config = tfma.EvalConfig(
    model_specs = [tfma.ModelSpec(label_key = 'price')],
    slicing_specs = [tfma.SlicingSpec()],
    metrics_specs = [
        tfma.MetricsSpec(metrics = [
                tfma.MetricConfig(class_name = 'ExampleCount'),
                tfma.MetricConfig(class_name = 'MeanAbsoluteError'),
                tfma.MetricConfig(class_name = 'MeanSquaredError')
        ])
    ]
)

In [28]:
from tfx.components import Evaluator

evaluator = Evaluator(
    examples = example_gen.outputs['examples'],
    model = trainer.outputs['model'],
    baseline_model = model_resolver.outputs['model'],
    eval_config = eval_config
)

interactive_context.run(evaluator)

ExecutionResult(
    component_id: Evaluator
    execution_id: 8
    outputs:
        evaluation: OutputChannel(artifact_type=ModelEvaluation, producer_component_id=Evaluator, output_key=evaluation, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        blessing: OutputChannel(artifact_type=ModelBlessing, producer_component_id=Evaluator, output_key=blessing, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

In [29]:
!pip install --upgrade tensorflow==2.10
!pip install --upgrade tensorflow-estimator==2.10
!pip install --upgrade tfx
!pip install --upgrade tensorflow_model_analysis
!pip install --upgrade fairness-indicators
!pip install tensorflow-model-analysis[addons] # Add "addons" extra requirements

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 578.0/578.0 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 41.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 44.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 40.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.7/438.7 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 51.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.3/781.3 kB 36.6 MB/s eta 0:00:00
  Attempting uninstall: keras
    Found existing installation: keras 3.5.0
    Uninstalling keras-3.5.0:
      Successfully uninstalled keras-3.5.0
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.15.0
    Uninstalling tensorflow-estimator-2.15.0:
      Successfully uninstalled tensorflow-estimator-2.15.0
  Attempting 

  Using cached tensorflow-2.16.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (4.2 kB)
  Using cached tensorboard-2.16.2-py3-none-any.whl.metadata (1.6 kB)
  Using cached protobuf-3.20.3-cp310-cp310-manylinux_2_12_x86_64.manylinux2010_x86_64.whl.metadata (679 bytes)
Using cached tensorflow-2.16.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (590.6 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 2.1 MB/s eta 0:00:00
Using cached tensorboard-2.16.2-py3-none-any.whl (5.5 MB)
Using cached protobuf-3.20.3-cp310-cp310-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (1.1 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 79.6 MB/s eta 0:00:00
  Attempting uninstall: tensorboard-data-server
    Found existing installation: tensorboard-data-server 0.6.1
    Uninstalling tensorboard-data-server-0.6.1:
      Successfully uninstalled tensorboard-data-server-0.6.1
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.19.6

  Using cached tensorflow_estimator-2.15.0-py2.py3-none-any.whl.metadata (1.3 kB)
INFO: pip is looking at multiple versions of tensorflow-data-validation to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of tf-keras to determine which version is compatible with other requirements. This could take a while.
  Using cached tf_keras-2.18.0-py3-none-any.whl.metadata (1.6 kB)
INFO: pip is looking at multiple versions of tensorflow-serving-api to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.2/475.2 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.0/19.0 MB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 77.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 81.7 MB/s eta 0:00

In [30]:
!pip install --upgrade tensorflow-estimator==2.15

# Pusher

In [31]:
from tfx.components import Pusher
from tfx.proto import pusher_pb2

pusher = Pusher(
    model = trainer.outputs['model'],
    model_blessing = evaluator.outputs['blessing'],
    push_destination = pusher_pb2.PushDestination(
        filesystem = pusher_pb2.PushDestination.Filesystem(
            base_directory = "serving_model_dir/humanStress-prediction-model"
        )
    )
)

interactive_context.run(pusher)

ExecutionResult(
    component_id: Pusher
    execution_id: 9
    outputs:
        pushed_model: OutputChannel(artifact_type=PushedModel, producer_component_id=Pusher, output_key=pushed_model, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))